# Predicting Mortgage Delinquency Risk

**Note: this is a new exercise, so if you find something weird, please bring it to my attention.**

You have been hired by a mortgage servicing firm (a company that buys mortgages and then collects mortgage payments from homeowners) to build a model to answer the question: 

**Given all available information about a newly issued mortgage, what is the likelihood that the mortgage will enter delinquency (the homeowner will be at least 30 days late on a mortgage payment) during the first two years of the mortgage?**

The servicer's hope, obviously, is to differentiate between mortgages to try and purchase (those that will be consistently paid) and mortgages they wish to avoid.

For this task, you have been given [REAL data on a sample of all US Standard single family home mortgages purchased or insured by Freddie Mac](https://www.freddiemac.com/research/datasets/sf-loanlevel-dataset) in a single calendar year along with payment data from that and two subsequent years.


## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_passive_prediction.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex2_merge_type",
    "ex4_num_mortgages",
    "ex5_num_obs",
    "ex7_num_mortgages",
    "ex7_share_delinquent",
    "ex10_num_obs",
    "ex12_roc_auc",
    "ex14_false_negative_rate",
    "ex16_num_obs",
    "ex16_share_delinquent",
    "ex17_false_negative_rate",
}
```


### Submission Limits

Please remember that you are **only allowed FOUR submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade Submissions that error out will **not** count against this total.

That's one more than usual in case there are issues with exercise clarity.

## Data Cleaning and Organization

Data for this exercise can be [found here](https://github.com/nickeubank/MIDS_Data/tree/master/mortgages/2004). This folder includes both the data to be used and documentation, though you can find [supplemental documentation here](https://www.freddiemac.com/research/datasets/sf-loanlevel-dataset).

The only difference between this data and the original Freddie Mac sampled data is that I've limited the scope of service data to three calendar years.

In [67]:
import warnings

warnings.filterwarnings("ignore")

results = {}

### Exercise 1

Begin by loading both: 

- the mortgage origination file (`sample_orig_2004.txt`). This *should* contain information on all mortgages issued in 2004, along with non-time varying features of these mortgages (the initial amount, the credit score of the applicant, etc.), and 
- the servicing data (`sample_svcg_2004orig_3years.txt`). This contains monthly records of all recorded payments (or non-payments) for all mortgages issued in 2004 during the calendar years of 2004, 2005, and 2006.

So the autograder can see the data, be sure to load it directly from a URL (don't download and load from your own system).

Load the data AND ensure your data has column names. You will likely need to reference the documentation to figure out how to do so.

In [68]:
import pandas as pd

origination_url = 'https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/mortgages/2004/sample_orig_2004.txt'
servicing_url = 'https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/mortgages/2004/sample_svcg_2004orig_3years.txt'

origination_columns = [ 'credit_score', '1st_payment_date','1st_time_buyer', 'maturity_date', 'msa', 'mi_percent', 
                        'num_of_units', 'occupancy_status', 'original_cltv', 'original_dti_ratio', 'original_upb', 
                        'original_ltv', 'original_interest_rt', 'channel', 'ppm_flag', 'amortization_type', 
                        'property_state', 'property_type', 'postal_code', 'loan_seq_num', 'loan_purpose', 
                        'original_loan_term', 'num_borrowers', 'seller_name', 'service_name', 'super_conforming', 
                        'prr_loan_seq_num', 'program_indicator', 'rr_indicator', 'prop_val_method', 'i/o_indicator', 
                        'mi_cancel_indicator']

servicing_columns = ['loan_seq_num', 'monthly_rep_per', 'current_actual_upb', 'current_delinq_status', 'loan_age', 
                     'month_to_maturity', 'defect_settle_date', 'mod_flag', 'zero_bal_code','zero_bal_eff_data', 
                     'curr_int_rate', 'curr_non_int_bearing_upb', 'ddlpi', 'mi_recoveries', 'net_sale_proceeds', 
                     'non_mi_recoveries', 'total_exp', 'legal_costs', 'maint_preserve_costs', 'taxes_insurance', 
                     'misc_exp', 'actual_loss_cal', 'cum_mod_cost', 'step_mod', 'payment_deferral', 'eltv', 
                     'zero_bal_rem_upb', 'delinq_acuired_int', 'disaster_delinq', 'borrower_assist_status', 
                     'curr_month_mod_cost', 'int_bearing_upb']

In [69]:
"""Loading the datasets"""
origination_data = pd.read_csv(origination_url, sep="|", names=origination_columns)

servicing_data = pd.read_csv(servicing_url, sep="|", names=servicing_columns)

print(origination_data.head())

   credit_score  1st_payment_date 1st_time_buyer  maturity_date      msa  \
0           701            200403              N         203402  45060.0   
1           648            200403              N         202402      NaN   
2           653            200403              Y         203402  20740.0   
3           747            200403              N         203402  30700.0   
4           731            200403              N         201902      NaN   

   mi_percent  num_of_units occupancy_status  original_cltv  \
0          25             1                P             90   
1           0             1                P             80   
2          17             1                P             87   
3           0             2                I             70   
4           0             1                P             80   

   original_dti_ratio  ...  num_borrowers                         seller_name  \
0                  43  ...              2                       Other sellers   
1 

In [70]:
print(servicing_data.head())

   loan_seq_num  monthly_rep_per  current_actual_upb current_delinq_status  \
0  F04Q10000027           200402            101000.0                     0   
1  F04Q10000027           200403            101000.0                     0   
2  F04Q10000027           200404            101000.0                     0   
3  F04Q10000027           200405            101000.0                     0   
4  F04Q10000027           200406            100000.0                     0   

   loan_age  month_to_maturity  defect_settle_date mod_flag  zero_bal_code  \
0         0                360                 NaN      NaN            NaN   
1         1                359                 NaN      NaN            NaN   
2         2                358                 NaN      NaN            NaN   
3         3                357                 NaN      NaN            NaN   
4         4                356                 NaN      NaN            NaN   

   zero_bal_eff_data  ...  cum_mod_cost  step_mod  payment_def

### Exercise 2

What is the unit of observation in `sample_orig_2004.txt` and in `sample_svcg_2004orig_3years.txt`?

**ANSWER**

* 1. In the **origination data file (`sample_orig_2004.txt`)** each row represents a unique mortgage loan at the time of origination. Therefore the unit observation is a loan.
* 2. The **servicing data file (`sample_svcg_2004orig_3years.txt`)** includes servicing data on the unit of observation mention in the previous paragraph. So it is a monthly record of a mortgage loan's payment status. Each row in this dataset represents specific information for a specific loan.

### Exercise 3

Merge your two datasets. Be sure to use the `validate` keyword argument in `merge`.

Assuming that you list the data associated with `sample_orig_2004.txt` first and `sample_svcg_2004orig_3years.txt` second, what keyword are you passing to `validate`? Store your answer as a string (use one of: `"1:1"`, `"m:1"`, `"1:m"`, `"m:m"`) in a dictionary called `results` under the key `ex2_merge_type`.

In [71]:
"""merging data as per instruction"""
merged_data = origination_data.merge(servicing_data, on='loan_seq_num', validate='1:m')

results = {'ex2_merge_type': '1:m'}

In [72]:
print(merged_data.head())

   credit_score  1st_payment_date 1st_time_buyer  maturity_date      msa  \
0           701            200403              N         203402  45060.0   
1           701            200403              N         203402  45060.0   
2           701            200403              N         203402  45060.0   
3           701            200403              N         203402  45060.0   
4           701            200403              N         203402  45060.0   

   mi_percent  num_of_units occupancy_status  original_cltv  \
0          25             1                P             90   
1          25             1                P             90   
2          25             1                P             90   
3          25             1                P             90   
4          25             1                P             90   

   original_dti_ratio  ...  cum_mod_cost  step_mod  payment_deferral eltv  \
0                  43  ...           NaN       NaN               NaN  NaN   
1         

### Exercise 4

Mortgages come in many shapes and flavors, however your servicer is only interested in predicting default for the more standard form of mortgage. Subset your data to only include:

- Mortgages taken out for purchase of a property,
- With first payments due in the quarter of origination or the first quarter after origination.

(In a perfect world we would just limit our analysis to mortgages where the first payment is due the month after origination. Unfortunately we only know the *quarter* of origination, so the only way to subset for relatively vanilla mortgages is to look for mortgages where the first payment was due in the same quarter or the quarter after origination.)

Subset for these mortgages. How many unique mortgages remain in the data? 

Hint: You may need to read the documentation for the `Loan Sequence Number` variable.

Store the resulting number of unique mortgages in `results` under the key `ex4_num_mortgages`.

In [73]:
""" Step 1. subset on purchase purpose """
purpose_subset = merged_data[merged_data['loan_purpose'] == 'P']

In [74]:
""" Step 2. job related to creating the new columns"""
# 1. Origination Year
purpose_subset['origination_year'] = '20' + purpose_subset['loan_seq_num'].str[1:3]

print(purpose_subset['origination_year'].unique())

['2004']


In [75]:
# 2. Origination Quarter
purpose_subset['origin_quarter'] = purpose_subset['loan_seq_num'].str[4].astype(int)

print(purpose_subset['origin_quarter'].unique())

[1 2 3 4]


In [76]:
# 4. Payment Year
purpose_subset['payment_year'] = purpose_subset['1st_payment_date'].astype(str).str[:4]

print(purpose_subset['payment_year'].unique())

['2004' '2005' '2006']


In [77]:
# 5. Payment Quarter
purpose_subset['1st_payment_date'] = purpose_subset['1st_payment_date'].astype(str)

def calculate_payment_quarter(month):
    if month in ['01', '02', '03']:
        return '1'
    elif month in ['04', '05', '06']:
        return '2'
    elif month in ['07', '08', '09']:
        return '3'
    elif month in ['10', '11', '12']:
        return '4'
    else:
        return None
    
purpose_subset['payment_quarter'] = purpose_subset['1st_payment_date'].str[-2:].apply(calculate_payment_quarter)

print(purpose_subset['payment_quarter'].unique())

['1' '2' '3' '4']


In [78]:
purpose_subset['payment_quarter'] = purpose_subset['payment_quarter'].astype(int)

In [79]:
""" Step 3. subsetting """

# Define the conditions
condition1 = (purpose_subset['origination_year'] == '2004') & \
             (purpose_subset['payment_year'] == '2004') & \
             (purpose_subset['origin_quarter'] == purpose_subset['payment_quarter'])

condition2 = (purpose_subset['origination_year'] == '2004') & \
             (purpose_subset['payment_year'] == '2004') & \
             (purpose_subset['origin_quarter'] + 1 == purpose_subset['payment_quarter'])

condition3 = (purpose_subset['origination_year'] == '2004') & \
             (purpose_subset['origin_quarter'] == 4) & \
             (purpose_subset['payment_year'] == '2005') & \
             (purpose_subset['payment_quarter'] == 1)

# Combine conditions to subset the DataFrame
mortgage_subset = purpose_subset[condition1 | condition2 | condition3]

In [80]:
""" Step 4. unique number"""
unique_mortgage = mortgage_subset['loan_seq_num'].nunique()
results['ex4_num_mortgages'] = unique_mortgage

print(f"Number of unique_mortgages for purchase with the first payments due on the quarter of origination or the quarter after is {unique_mortgage}")
print(results)

Number of unique_mortgages for purchase with the first payments due on the quarter of origination or the quarter after is 17504
{'ex2_merge_type': '1:m', 'ex4_num_mortgages': 17504}


### Exercise 5

The servicer wants to predict delinquency during the first 24 payment due dates (you may assume payments are due every month starting with the month the first payment is due). Subset the data to these first 24 (possible) payment due dates.

Note that not all loans will have 24 records in the servicing file in the first 24 months as a result of data merging issues on behalf of Freddie Mac. As noted in the Freddie Mac documentation:

> For a given loan, each monthly reporting period in the monthly performance data file combines data elements from multiple reporting cycles and systems at Freddie Mac. As such, perceived data anomalies may be a result of timing mismatches between default/delinquency reporting cycles and investor reporting cycles. Examples of some commonly occurring anomalies in the data are included throughout this section. In all cases, the best information available at the time the Dataset is generated, subject to operational constraints, is used.

So subset for the first two years of (possible) payments, resulting in *up to* 24 observations per mortgage (but potentially less given the data cleanliness issues).

After this subsetting, store the number of remaining observations (not mortgages, observation) in `results` under the key `"ex5_num_obs"`.

In [83]:
"""I will use 'monthly_rep_per' for extracting loans with first 24 payments"""

mortgage_subset['monthly_rep_per'] = pd.to_datetime(mortgage_subset['monthly_rep_per'], format='%Y%m')
mortgage_subset["1st_payment_date"] = pd.to_datetime(mortgage_subset["1st_payment_date"], format="%Y%m")
mortgage_subset["months_since_1st_payment"] = (mortgage_subset["monthly_rep_per"] - mortgage_subset["1st_payment_date"]).dt.days // 30

two_year_payment = mortgage_subset[
    (mortgage_subset["months_since_1st_payment"] >= 0)
    & (mortgage_subset["months_since_1st_payment"] <= 23)
]

ex5_num_obs = len(two_year_payment)
results['ex5_num_obs'] = ex5_num_obs

print(results)

{'ex2_merge_type': '1:m', 'ex4_num_mortgages': 17504, 'ex5_num_obs': 379461}


### Exercise 6

For each unique mortgage in your dataset, create an indicator variable that takes on a value of 1 if, at any time during this period, the mortgage has been delinquent.

Delinquency status is stored in the variable `CURRENT LOAN DELINQUENCY STATUS`, and is coded as:

> CURRENT LOAN DELINQUENCY STATUS – A value corresponding to the number of days the borrower is delinquent, based on the due date of last paid installment (“DDLPI”) reported by servicers to Freddie Mac, and is calculated under the Mortgage Bankers Association (MBA) method.
If a loan has been acquired by REO, then the Current Loan Delinquency Status will reflect the value corresponding to that status (instead of the value corresponding to the number of days the borrower is delinquent).
>
> 0 = Current, or less than 30 days delinquent
> 
> 1 = 30-59 days delinquent
> 
> 2=60–89days delinquent
> 
> 3=90–119days delinquent
> 
> And so on...
> 
> RA = REO Acquisition


In [88]:
# first let's convert everything to string so we can save 
## some area for condition writing in code

two_year_payment['current_delinq_status'] = two_year_payment['current_delinq_status'].astype(str)
two_year_payment['delinq_indicator'] = two_year_payment['current_delinq_status'].apply(lambda x: 0 if x == '0' else 1)
two_year_payment

,credit_score,1st_payment_date,1st_time_buyer,maturity_date,msa,mi_percent,num_of_units,occupancy_status,original_cltv,original_dti_ratio,...,disaster_delinq,borrower_assist_status,curr_month_mod_cost,int_bearing_upb,origination_year,origin_quarter,payment_year,payment_quarter,months_since_1st_payment,delinq_indicator
71,653,2004-03-01,Y,203402,20740.0,17,1,P,87,25,...,NaN,NaN,NaN,126000.00,2004,1,2004,1,0,0
72,653,2004-03-01,Y,203402,20740.0,17,1,P,87,25,...,NaN,NaN,NaN,126000.00,2004,1,2004,1,1,0
73,653,2004-03-01,Y,203402,20740.0,17,1,P,87,25,...,NaN,NaN,NaN,126000.00,2004,1,2004,1,2,0
74,653,2004-03-01,Y,203402,20740.0,17,1,P,87,25,...,NaN,NaN,NaN,126000.00,2004,1,2004,1,3,0
75,653,2004-03-01,Y,203402,20740.0,17,1,P,87,25,...,NaN,NaN,NaN,126000.00,2004,1,2004,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287129,805,2004-12-01,N,203411,17140.0,12,1,P,85,33,...,NaN,NaN,NaN,219680.66,2004,4,2004,4,23,0
1287131,736,2004-12-01,N,201911,NaN,0,1,S,56,7,...,NaN,NaN,NaN,298523.18,2004,4,2004,4,21,0
1287132,736,2004-12-01,N,201911,NaN,0,1,S,56,7,...,NaN,NaN,NaN,297039.44,2004,4,2004,4,22,0
1287133,736,2004-12-01,N,201911,NaN,0,1,S,56,7,...,NaN,NaN,NaN,295548.74,2004,4,2004,4,23,0


### Exercise 7

At this point, you should be able to drop all servicing variables reported on a monthly basis and just keep information about the original mortgage issuance (and still keep an indicator for whether the mortgage has ever been delinquent).

Store the final number of mortgages in your data under `ex7_num_mortgages` and the share (between 0 and 1) of mortgages that have been delinquent under `ex7_share_delinquent`.

In [89]:
"""let's keep columns from origination data file"""

# Step 1. drop duplicates
columns_to_keep = [ 'credit_score', '1st_payment_date','1st_time_buyer', 'maturity_date', 'msa', 'mi_percent', 
                    'num_of_units', 'occupancy_status', 'original_cltv', 'original_dti_ratio', 'original_upb', 
                    'original_ltv', 'original_interest_rt', 'channel', 'ppm_flag', 'amortization_type', 
                    'property_state', 'property_type', 'postal_code', 'loan_seq_num', 'loan_purpose', 
                    'original_loan_term', 'num_borrowers', 'seller_name', 'service_name', 'super_conforming', 
                    'prr_loan_seq_num', 'program_indicator', 'rr_indicator', 'prop_val_method', 'i/o_indicator', 
                    'mi_cancel_indicator', 'delinq_indicator']
final_dataset = two_year_payment[columns_to_keep].drop_duplicates()

18709


In [92]:
ex7_num_mortgages = final_dataset['loan_seq_num'].nunique()
results['ex7_num_mortgages'] = ex7_num_mortgages

ex7_share_delinquent = final_dataset['delinq_indicator'].mean()
ex7_share_delinquent

0.06638516222139078

## Modelling Delinquency Risk

Your data should now be relatively [tidy](https://vita.had.co.nz/papers/tidy-data.pdf), in the technical sense of the term. And that means it should be relatively straightforward for you to build a model that answers the question "Given the features of a newly originated mortgage, how likely is the mortgage holder to fall into delinquency within the first two years after origination?"

### Exercise 8

First, we need to identify the target for our model useful predictors from the data and do feature engineering.

Let's begin with identifying some features that probably *aren't* going to be useful. For example, `"Metropolitan Statistical Area (MSA) Or Metropolitan Division"` is probably *not* an appropriate feature to include in this analysis. Can you figure out why? Make sure to show (quantitatively) why not. 

Hint: should be more than the missing rate.

Hint 2: how many observations for a given city do you think you'd need to determine if that city had especially high mortgage delinquency rates?

Hint 3: if not all possible values of a variable are present in your training data, what problem might that cause during testing and deployment?


### Exercise 9

For your analysis, include the following variables: 

```
Credit Score
First Time Homebuyer Flag
Number of Units
Mortgage Insurance Percentage (MI %)
Occupancy Status
Original Debt-to-Income (DTI) Ratio
Original UPB
Original Loan-to-Value (LTV)
Original Interest Rate
Channel
Prepayment Penalty Mortgage (PPM) Flag
Amortization Type (Formerly Product Type)
Property State
Property Type
Original Loan Term
Number of Borrowers
Interest Only (I/O) Indicator
```

Be sure to clean these variables. When doing so, please treat missing data as missing (e.g., `np.nan`, not as a distinct category).

### Exercise 10

The next step in our analysis is to convert our categorical variables to one-hot-encodings and use `train_test_split` to split our data.

To ensure replicability, **before** you `train_test_split` your data, please sort your data by `Loan Sequence Number`. This will ensure when we split the data with a random seed below, everyone will get the same split and the autograder will function.

You may create your one-hot-encodings however you wish, but I'm a fan of the [patsy library's](https://patsy.readthedocs.io/en/latest/overview.html) `dmatrices` function.

Hint: You should end up with 8 categorical variables, including some binary flags and `Number_of_Borrowers`, `Number_of_Units` (which you could argue should be continuous, but I think are better treated as categorical).

Store the number of observations in your final dataset in `ex10_num_obs`.

### Exercise 11

Use `train_test_split` from `sklearn.model_selection` to split the data. 

Before you do, Use `0.2` as the `test_size` and use `random_state=42`.

### Exercise 12

Now fit a `GradientBoostingClassifier` to the data (from `sklearn.ensemble`). Set `random_state=42`. using `roc_auc_score`, get your ROC AUC score against the test data. Store in `results` under the key `"ex12_roc_auc"`.

### Exercise 13

Use the `predict` method to generate a confusion matrix. What problem do you see with the result?

### Exercise 14

To address the problem from Exercise 13, use `.predict_proba()` to set your own threshold for classification. Your stakeholder is mostly concerned with False Negatives (mortgages classified as safe that actually are not), so use a 8% probability threshold to get a good balance of a low False Negative rate with a reasonable amount of mortgages still being considered "viable."

What is the False Negative rate at an 8% classification threshold from the model above?

Store the result under the key `"ex14_false_negative_rate"`.

### Exercise 15

Your stakeholder wants to by as many mortgages as it can while maintaining a delinquency rate of purchased mortgages below 5%. Based on your answer above, do you feel like your model can provide that level of performance?

## Now To The Future

The preceding analysis is precisely the type of analysis you would do if, in late 2006, you'd been asked to evaluate mortgage performance in the last two years for use going forward. So let's see how your model performs now!

In this [folder](https://github.com/nickeubank/MIDS_Data/tree/master/mortgages/2007) you will find data on mortgages originated in 2007 along with servicing data from 2007, 2008, and 2009.

### Exercise 16

Please load this data (again, from a URL to help the autograder) and clean it in the same manner as before. As a sanity check, how many observations do you have in the final dataset (after you've removed observations with missing values to allow you to generate predicted delinquency rates)? 

Store the final number of observations in `"ex16_num_obs"` and the share of those mortgages that are delinquent in `"ex16_share_delinquent"`


### Exercise 17




Had your stakeholder purchased mortgages using your model, what would the resulting False Negative rate have been? (e.g., compare the predicted values for mortgages using the model trained above with realized outcomes). Store your result under the key `"ex17_false_negative_rate"`


### Exercise 18

How did the retrospective performance of your model compare to its actual performance moving forward? Why? Did you stay below the 5% target for False Negatives set by the stakeholder?